In [35]:
## Imports

import numpy as np 
import math
import time
from math import exp, log, pi, sqrt
from scipy import stats
import yahoo_fin
from yahoo_fin import options
from yahoo_fin import stock_info
import pandas_datareader
from pandas_datareader import data as pdr
from datetime import datetime

In [5]:
class OptionsPricing:

    def __init__(self, S, K, T, r, q, sigma):
        self.S = S
        self.K = K
        self.T = T
        self.r = r
        self.q = q 
        self.sigma = sigma 

#================================ CALLS ====================================================================

    def call_price(self):           # Return the theoretical value of the call option

        d1 = (np.log(S/K) + T * (r - q + sigma ** 2 / 2)) / (sigma * sqrt(T))
        d2 = d1 - sigma * sqrt(T)

        price = S * np.exp(-q * T) * stats.norm.cdf(d1) - K * np.exp(-r * T) * stats.norm.cdf(d2)

        return price

    def call_delta(self):

        d1 = (np.log(S/K) + T * (r - q + sigma ** 2 / 2)) / (sigma * sqrt(T))
        d2 = d1 - sigma * sqrt(T)

        delta = stats.norm.cdf(d1) * exp(-q * T)

        return delta

    def call_gamma(self):

        d1 = (np.log(S/K) + T * (r - q + sigma ** 2 / 2)) / (sigma * sqrt(T))
        d2 = d1 - sigma * sqrt(T)

        gamma = (exp(-1* d1 ** 2) / 2) / sqrt(2 * pi)

        return gamma

    def call_theta(self):

        d1 = (np.log(S/K) + T * (r - q + sigma ** 2 / 2)) / (sigma * sqrt(T))
        d2 = d1 - sigma * sqrt(T)

        theta = (-(S * exp(-1 * (d1 ** 2) / 2) / sqrt(2 * pi) * sigma * exp(-q * T)/ (2 * sqrt(T))) - (r * (K * exp(-r * T)) * stats.norm.cdf(d2)) + (q * S * stats.norm.cdf(d1) * exp(-q * T))) / 365      #Theta defined as the loss of value every calendar day (not trading day)

        return theta

    def call_vega(self):

        d1 = (np.log(S/K) + T * (r - q + sigma ** 2 / 2)) / (sigma * sqrt(T))
        d2 = d1 - sigma * sqrt(T)

        vega = exp(-1 * d1 ** 2 / 2) / sqrt(2 * pi) * exp(-q * T) * S * sqrt(T) / 100

        return vega
    
    def call_rho(self):

        d1 = (np.log(S/K) + T * (r - q + sigma ** 2 / 2)) / (sigma * sqrt(T))
        d2 = d1 - sigma * sqrt(T)

        rho = K * T * exp(-r * T) * stats.norm.cdf(d2) / 100  

        return rho


#================================ PUTS ====================================================================

    def put_price(self):            # Return the theoretical value of the put option

        d1 = (np.log(S/K) + T * (r - q + sigma ** 2 / 2)) / (sigma * sqrt(T))
        d2 = d1 - sigma * sqrt(T)

        price = K * np.exp(-r * T) * stats.norm.cdf(-d2) - S * np.exp(-q * T) * stats.norm.cdf(-d1)

        return price

    def put_delta(self):

        d1 = (np.log(S/K) + T * (r - q + sigma ** 2 / 2)) / (sigma * sqrt(T))
        d2 = d1 - sigma * sqrt(T)

        delta = (stats.norm.cdf(d1)-1) * exp(-q * T)

        return delta

    def put_gamma(self):

        d1 = (np.log(S/K) + T * (r - q + sigma ** 2 / 2)) / (sigma * sqrt(T))
        d2 = d1 - sigma * sqrt(T)

        gamma = (exp(-1* d1 ** 2) / 2) / sqrt(2 * pi)

        return gamma

    def put_theta(self):

        d1 = (np.log(S/K) + T * (r - q + sigma ** 2 / 2)) / (sigma * sqrt(T))
        d2 = d1 - sigma * sqrt(T)

        theta = (-(S * exp(-1 * d1 ** 2 / 2) / sqrt(2 * pi) * sigma * exp(-q * T) / (2 * sqrt(T))) + (r * K * exp(-r * T) * stats.norm.cdf(-d2)) - (q * S * stats.norm.cdf(-d1) * exp(-q * T))) / 365         # Theta defined as the loss of value every calendar day (365), not every trading day

        return theta

    def put_vega(self):

        d1 = (np.log(S/K) + T * (r - q + sigma ** 2 / 2)) / (sigma * sqrt(T))
        d2 = d1 - sigma * sqrt(T)

        vega = exp(-1 * d1 ** 2 / 2) / sqrt(2 * pi) * exp(-q * T) * S * sqrt(T) / 100

        return vega

    def put_rho(self):

        d1 = (np.log(S/K) + T * (r - q + sigma ** 2 / 2)) / (sigma * sqrt(T))
        d2 = d1 - sigma * sqrt(T)

        rho = -K * T * exp(-r * T) * stats.norm.cdf(-d2) / 100

        return rho


## Input Metrics of the Option

In [6]:
ticker = 'AAPL'
price = stock_info.get_live_price(f"{ticker}")           # Underlying price
S = round(price, 2)
S

/home/nate/development/env/lib/python3.11/site-packages/yahoo_fin/stock_info.py:580: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return df.close[-1]


194.53

In [7]:
option = 'put'.upper()    #Determine which option type

if option == 'PUT':
        chain = options.get_puts(f"{ticker}")

elif option == 'CALL':
        chain = options.get_calls(f"{ticker}")

print(chain['Strike'])

0      65.0
1      70.0
2      75.0
3      80.0
4      85.0
5      90.0
6      95.0
7     100.0
8     105.0
9     110.0
10    115.0
11    120.0
12    125.0
13    130.0
14    135.0
15    140.0
16    145.0
17    150.0
18    152.5
19    155.0
20    157.5
21    160.0
22    162.5
23    165.0
24    167.5
25    170.0
26    172.5
27    175.0
28    177.5
29    180.0
30    182.5
31    185.0
32    187.5
33    190.0
34    192.5
35    195.0
36    197.5
37    200.0
38    202.5
39    205.0
40    207.5
41    210.0
42    212.5
43    215.0
44    217.5
45    220.0
46    225.0
47    230.0
48    235.0
49    240.0
50    245.0
51    250.0
52    255.0
53    260.0
54    270.0
55    275.0
56    280.0
57    285.0
58    290.0
59    295.0
Name: Strike, dtype: float64


/home/nate/development/env/lib/python3.11/site-packages/yahoo_fin/options.py:42: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(requests.get(site, headers=headers).text)


In [8]:
K = float(input("Select a Strike Price: "))     # Strike Price

K

165.0

In [24]:
dates = options.get_expiration_dates(f"{ticker}")
print("\nSelect an Expiry Date\n", dates)


Select an Expiry Date
 ['December 15, 2023', 'December 22, 2023', 'December 29, 2023', 'January 5, 2024', 'January 12, 2024', 'January 19, 2024', 'January 26, 2024', 'February 16, 2024', 'March 15, 2024', 'April 19, 2024', 'June 21, 2024', 'July 19, 2024', 'September 20, 2024', 'December 20, 2024', 'January 17, 2025', 'June 20, 2025', 'September 19, 2025', 'December 19, 2025', 'January 16, 2026']


In [26]:
date1 = input("\nExpiry Date (MM/DD/YY): ")
date_str = date1
date_object = datetime.strptime(date_str, '%m/%d/%y').date()
    #print(type(date_object))
    #print(date_object)

from datetime import date
today = date.today()
td = date_object - today
print(f"\nExpiry Date: {date1}")
print(f"\nDays to Expiry: {td.days}")

T = td.days / 365



Expiry Date: 01/24/24

Days to Expiry: 43


In [9]:
r = stock_info.get_live_price("^IRX")
r = round(r, 2)/100
r

/home/nate/development/env/lib/python3.11/site-packages/yahoo_fin/stock_info.py:580: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return df.close[-1]


0.0525

In [12]:
div = stock_info.get_dividends(ticker, "01-01-2023")
i = len(div)
q = i * div['dividend'][i-1] / S
q                                                   # annual dividend yield


/home/nate/development/env/lib/python3.11/site-packages/yahoo_fin/stock_info.py:693: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  frame.index = pd.to_datetime(frame.index, unit = "s")
/tmp/ipykernel_1975/3436392140.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  q = i * div['dividend'][i-1] / S


0.004934971469696191

In [32]:
from dateutil.relativedelta import relativedelta
minusoneyear = today - relativedelta(years=1)

df = stock_info.get_data(f"{ticker}", start_date= minusoneyear, end_date= today)
df['close']

log_returns = np.log(df.close/df.close.shift(1)).dropna()
daily_std = log_returns.std()
annualized_vol = daily_std * np.sqrt(252)
sigma = round(annualized_vol, 4)
sigma 

0.2167

In [33]:
model = OptionsPricing(S, K, T, r, q, sigma)

callprice = model.call_price()
calldelta = model.call_delta()
callgamma = model.call_gamma()
calltheta = model.call_theta()
callvega = model.call_vega()
callrho = model.call_rho()

putprice = model.put_price()
putdelta = model.put_delta()
putgamma = model.call_gamma()
puttheta = model.put_theta()
putvega = model.put_vega()
putrho = model.put_rho()


In [34]:
callprice

30.484505110518626